# Bloque 6 Miografía: Práctica 8 - Procesado de Señal aplicado al EMG

En esta práctica, nos adentraremos en el **procesado de señal aplicado al electromiograma (EMG)**, centrándonos en los principales métodos paramétricos para la estimación de la **Densidad Espectral de Potencia (PSD)**. La PSD es una herramienta fundamental en el análisis de señales biomédicas, ya que nos permite entender cómo se distribuye la potencia de una señal en función de la frecuencia.

Durante esta sesión, exploraremos diversas técnicas paramétricas que nos ayudarán a estimar la PSD de señales EMG, proporcionando una visión más profunda y detallada de la actividad muscular. Estas técnicas incluyen métodos como el **modelo autoregresivo (AR)**, el **modelo de media móvil (MA)** y el **modelo autoregresivo de media móvil (ARMA)**.

El objetivo principal de esta práctica es familiarizarnos con estos métodos y aprender a aplicarlos correctamente para obtener estimaciones precisas y útiles de la PSD en señales EMG. Al finalizar, deberíamos ser capaces de interpretar los resultados y comprender las ventajas y limitaciones de cada método.

## 1- Modelos Autoregresivos (AR)

Considere una señal del tipo $x(n) = -0.9x(n-2)+w(n)$ donde $w$ es ruido blanco gaussiano de media 0 y desviación estándar unitaria. Obtenga su espectro mediante un modelo AR y el método de Autocorrelación para los siguientes ordenes $p=2,4,12$. Ayúdese de la función from statsmodels.regression.linear_model import yule_walker

In [9]:
import numpy as np
import plotly.graph_objects as go
from statsmodels.regression.linear_model import yule_walker
from scipy import signal
from scipy.signal import freqz
import spectrum

# Generar la señal x(n)
np.random.seed(0)  # Fijar la semilla para reproducibilidad
n = np.arange(0, 66)  # Crear un array de 0 a 65
w = np.random.normal(0, 1, len(n))  # Generar ruido blanco
x = w  # Asignar el ruido blanco a x

# Coeficientes del filtro AR
aCoef = np.array([1, -0.9])  # Coeficientes arbitrarios para el filtro AR (ejemplo)
x = signal.lfilter(aCoef, 1, x)  # Aplicar el filtro AR a la señal x

# yulewalker
p = spectrum.pyule(x, 2, norm="biased", NFFT=1024)  # Estimar el PSD usando el método de Yule-Walker con p=2
psd_yw2 = p.psd  # Obtener el PSD estimado
w = np.linspace(0, np.pi, 512)  # Crear un array de frecuencias

p = spectrum.pyule(x, 4, norm="biased", NFFT=1024)  # Estimar el PSD usando el método de Yule-Walker con p=4
psd_yw4 = p.psd  # Obtener el PSD estimado

p = spectrum.pyule(x, 12, norm="biased", NFFT=1024)  # Estimar el PSD usando el método de Yule-Walker con p=12
psd_yw12 = p.psd  # Obtener el PSD estimado

# Crear la figura con Plotly
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=w,
        y=10 * np.log10(psd_yw2),  # Convertir el PSD a dB
        mode="lines",
        name="p=2",
    )
)

fig.add_trace(
    go.Scatter(
        x=w,
        y=10 * np.log10(psd_yw4),  # Convertir el PSD a dB
        mode="lines",
        name="p=4",
    )
)

fig.add_trace(
    go.Scatter(
        x=w,
        y=10 * np.log10(psd_yw12),  # Convertir el PSD a dB
        mode="lines",
        name="p=12",
    )
)

fig.update_layout(
    title="PSD de la señal x(n) utilizando diferentes métodos AR",
    xaxis_title="Frecuencia (Hz)",
    yaxis_title="PSD (dB)",
    legend_title="Método",
)

fig.show()  # Mostrar la figura

## Comparación de distintos métodos de estimación para los modelos AR

Considere una señal del tipo $x(n) = 2.7377x(n-1)-3.7476x(n-2)+2.6293x(n-3)-0.9224x(n-4)+w(n)$ donde $w$ es ruido blanco gaussiano de media 0 y desviación estándar unitaria. Obtenga su espectro mediante un modelo AR ayúdese del apartado anterior y utilize los códigos proporcionados en auxiliar. Use un N de 128 y haga 50 repeticiones, dibuje todas las repeticiones y la media, use orden 4 en todos los casos

In [10]:
import numpy as np  # Importar la biblioteca numpy para manejo de arrays y funciones matemáticas
import plotly.graph_objects as go  # Importar plotly para crear gráficos interactivos
from scipy import signal  # Importar signal de scipy para procesamiento de señales
from plotly.subplots import make_subplots  # Importar make_subplots para crear subgráficos

# Generar la señal x(n)
np.random.seed(120)  # Fijar la semilla para reproducibilidad
n = np.arange(0, 132)  # Crear un array de 0 a 131
w = np.random.normal(0, 1, (len(n), 50))  # Generar ruido blanco con media 0 y desviación estándar 1 para 50 realizaciones
x = w  # Asignar el ruido blanco a x
aCoef = [1, -2.7377, 3.7476, -2.6293, 0.9224] # Coeficientes del filtro AR (AutoRegresivo)
z = signal.lfilter([1], aCoef, w) # Aplicar el filtro AR a la señal x
x = z  # Asignar la señal filtrada a x
order = 5  # Orden del modelo AR

# Inicializar matrices para almacenar las PSD (Densidad Espectral de Potencia)
psdYW = np.zeros((len(n)//2 + 1, 50))  # Ajustamos el tamaño
psdCov = np.zeros((len(n)//2 + 1, 50))
psdMcov = np.zeros((len(n)//2 + 1, 50))
psdBurg = np.zeros((len(n)//2 + 1, 50))  

# Calcular los coeficientes AR y sigma para cada realización
for i in range(50):
    p = spectrum.pyule(x[:, i], order)  # Estimar el PSD usando el método de Yule-Walker
    psdYW[:, i] = p.psd[:len(n)//2 + 1]

    p = spectrum.pcovar(x[:, i], order)  # Estimar el PSD usando el método de Covarianza
    psdCov[:, i] = p.psd[:len(n)//2 + 1]

    p = spectrum.pmodcovar(x[:, i], order)  # Estimar el PSD usando el método de Covarianza Modificada
    psdMcov[:, i] = p.psd[:len(n)//2 + 1]

    p = spectrum.pburg(x[:, i], order)  # Estimar el PSD usando el método de Burg
    psdBurg[:, i] = p.psd[:len(n)//2 + 1]

# Calcular las PSD medias
psd_yw_mean = np.mean(psdYW, axis=1)  # Media de las PSD estimadas por Yule-Walker
psd_cv_mean = np.mean(psdCov, axis=1)  # Media de las PSD estimadas por Covarianza
psd_mcv_mean = np.mean(psdMcov, axis=1)  # Media de las PSD estimadas por Covarianza Modificada
psd_b_mean = np.mean(psdBurg, axis=1)  # Media de las PSD estimadas por Burg

# Frecuencia
freqs = np.linspace(0, 1, len(n)//2 + 1)  # Crear un array de frecuencias de 0 a 1 ajustado

# Crear la figura con Plotly
fig = make_subplots(
    rows=2,  # Número de filas de subgráficos
    cols=2,  # Número de columnas de subgráficos
    subplot_titles=("YW", "Covarianza", "Covarianza Modificada", "Burg"),  # Títulos de los subgráficos
)

# Añadir trazas para cada método
for i in range(50):
    fig.add_trace(
        go.Scatter(
            x=freqs,  # Asignar el array de frecuencias al eje x
            y=psdYW[:, i],  # Convertir el PSD a dB y asignarlo al eje y
            mode="lines",  # Modo de trazado en líneas
            name=f"Realización {i+1}",  # Nombre de la serie
            showlegend=False,  # No mostrar la leyenda para estas trazas
            line=dict(color="blue", width=0.1),  # Estilo de línea
        ),
        row=1,  # Fila del subgráfico
        col=1,  # Columna del subgráfico
    )
    fig.add_trace(
        go.Scatter(
            x=freqs,  # Asignar el array de frecuencias al eje x
            y=psdCov[:, i],  # Convertir el PSD a dB y asignarlo al eje y
            mode="lines",  # Modo de trazado en líneas
            name=f"Realización {i+1}",  # Nombre de la serie
            showlegend=False,  # No mostrar la leyenda para estas trazas
            line=dict(color="red", width=0.1),  # Estilo de línea
        ),
        row=1,  # Fila del subgráfico
        col=2,  # Columna del subgráfico
    )
    fig.add_trace(
        go.Scatter(
            x=freqs,  # Asignar el array de frecuencias al eje x
            y=psdMcov[:, i],  # Convertir el PSD a dB y asignarlo al eje y
            mode="lines",  # Modo de trazado en líneas
            name=f"Realización {i+1}",  # Nombre de la serie
            showlegend=False,  # No mostrar la leyenda para estas trazas
            line=dict(color="green", width=0.1),  # Estilo de línea
        ),
        row=2,  # Fila del subgráfico
        col=1,  # Columna del subgráfico
    )
    fig.add_trace(
        go.Scatter(
            x=freqs,  # Asignar el array de frecuencias al eje x
            y=psdBurg[:, i],  # Convertir el PSD a dB y asignarlo al eje y
            mode="lines",  # Modo de trazado en líneas
            name=f"Realización {i+1}",  # Nombre de la serie
            showlegend=False,  # No mostrar la leyenda para estas trazas
            line=dict(color="purple", width=0.1),  # Estilo de línea
        ),
        row=2,  # Fila del subgráfico
        col=2,  # Columna del subgráfico
    )

# Añadir trazas para las PSD medias
fig.add_trace(
    go.Scatter(
        x=freqs,  # Asignar el array de frecuencias al eje x
        y=psd_yw_mean,  # Convertir el PSD a dB y asignarlo al eje y
        mode="lines",  # Modo de trazado en líneas
        name="Media YW",  # Nombre de la serie
        line=dict(color="blue", width=2),  # Estilo de línea
    ),
    row=1,  # Fila del subgráfico
    col=1,  # Columna del subgráfico
)
fig.add_trace(
    go.Scatter(
        x=freqs,  # Asignar el array de frecuencias al eje x
        y=psd_cv_mean,  # Convertir el PSD a dB y asignarlo al eje y
        mode="lines",  # Modo de trazado en líneas
        name="Media Covarianza",  # Nombre de la serie
        line=dict(color="red", width=2),  # Estilo de línea
    ),
    row=1,  # Fila del subgráfico
    col=2,  # Columna del subgráfico
)
fig.add_trace(
    go.Scatter(
        x=freqs,  # Asignar el array de frecuencias al eje x
        y=psd_mcv_mean,  # Convertir el PSD a dB y asignarlo al eje y
        mode="lines",  # Modo de trazado en líneas
        name="Media Covarianza Modificada",  # Nombre de la serie
        line=dict(color="green", width=2),  # Estilo de línea
    ),
    row=2,  # Fila del subgráfico
    col=1,  # Columna del subgráfico
)
fig.add_trace(
    go.Scatter(
        x=freqs,  # Asignar el array de frecuencias al eje x
        y=psd_b_mean,  # Convertir el PSD a dB y asignarlo al eje y
        mode="lines",  # Modo de trazado en líneas
        name="Media Burg",  # Nombre de la serie
        line=dict(color="purple", width=2),  # Estilo de línea
    ),
    row=2,  # Fila del subgráfico
    col=2,  # Columna del subgráfico
)

# Actualizar el diseño de la figura
fig.update_layout(
    title="PSD de la señal x(n) utilizando diferentes métodos AR",  # Título del gráfico
    xaxis_title="Frecuencia (Hz)",  # Título del eje x
    yaxis_title="PSD (dB)",  # Título del eje y
    legend_title="Método",  # Título de la leyenda
)

fig.show()  # Mostrar la figura

### Selección del orden óptimo

Para el caso anterior, elija el orden óptimo el método de Yule-Walker y el de Burg mediante los siguientes criterios:
- Criterio de información de Akike (spectrum.AIC)
- Criterio de la longitud mínima de descripción (spectrum.MDL)
- Criterio del error de predicción final de Akike (spectrum.FPE)

El valor de $\epsilon_p$ viene dado por la potencia del ruido blanco, que se obtiene como 2º parámetro en las estimaciones de los parámetros AR. Podéis consultar la ayuda en [spectrum](https://pyspectrum.readthedocs.io/)



In [11]:
import numpy as np  # Importar la biblioteca numpy para manejo de arrays y funciones matemáticas
import plotly.graph_objects as go  # Importar plotly para crear gráficos interactivos
from scipy import signal  # Importar signal de scipy para procesamiento de señales
from plotly.subplots import make_subplots  # Importar make_subplots para crear subgráficos
import spectrum  # Importar la biblioteca spectrum para análisis espectral

# Generar la señal x(n)
np.random.seed(120)  # Fijar la semilla para reproducibilidad
n = np.arange(0, 132)  # Crear un array de 0 a 131
w = np.random.normal(0, 1, len(n))  # Generar ruido blanco con media 0 y desviación estándar 1
x = w  # Asignar el ruido blanco a x
aCoef = [1, -2.7377, 3.7476, -2.6293, 0.9224] # Coeficientes del filtro AR (AutoRegresivo)
z = signal.lfilter([1], aCoef, w)  # Aplicar el filtro AR a la señal x

# Inicializar listas para almacenar los valores de AIC, C y FPE
AIC = []
C = []
FPE = []

# Calcular los coeficientes AR y los criterios para cada orden de modelo
for o in range(15):
    a, rho, _ = spectrum.aryule(x, o)  # Estimar los coeficientes AR usando el método de Yule-Walker
    N = len(x)
    AIC.append(spectrum.AIC(N, rho, o))  # Calcular el criterio AIC
    C.append(spectrum.MDL(N, rho, o))  # Calcular el criterio MDL
    FPE.append(spectrum.FPE(N, rho, o))  # Calcular el criterio FPE

# Encontrar las posiciones de los valores mínimos
min_AIC_pos = AIC.index(min(AIC))
min_C_pos = C.index(min(C))
min_FPE_pos = FPE.index(min(FPE))

# Crear subgráficos
fig = make_subplots(rows=1, cols=3, subplot_titles=("AIC", "C", "FPE"))

# Añadir trazas para AIC
fig.add_trace(
    go.Scatter(x=list(range(1, 16)), y=AIC, mode="lines+markers", name="AIC"),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=[min_AIC_pos + 1],
        y=[AIC[min_AIC_pos]],
        mode="markers",
        marker=dict(color="red", size=10),
        name="Min AIC",
    ),
    row=1,
    col=1,
)

# Añadir trazas para C
fig.add_trace(
    go.Scatter(x=list(range(1, 16)), y=C, mode="lines+markers", name="C"), row=1, col=2
)
fig.add_trace(
    go.Scatter(
        x=[min_C_pos + 1],
        y=[C[min_C_pos]],
        mode="markers",
        marker=dict(color="green", size=10),
        name="Min C",
    ),
    row=1,
    col=2,
)

# Añadir trazas para FPE
fig.add_trace(
    go.Scatter(x=list(range(1, 16)), y=FPE, mode="lines+markers", name="FPE"),
    row=1,
    col=3,
)
fig.add_trace(
    go.Scatter(
        x=[min_FPE_pos + 1],
        y=[FPE[min_FPE_pos]],
        mode="markers",
        marker=dict(color="blue", size=10),
        name="Min FPE",
    ),
    row=1,
    col=3,
)

# Actualizar el diseño
fig.update_layout(
    title="AIC, C y FPE Yule-Walker", xaxis_title="Orden", yaxis_title="Valor"
)

# Mostrar el gráfico
fig.show()

# Generar la señal x(n) nuevamente
np.random.seed(120)  # Fijar la semilla para reproducibilidad
n = np.arange(0, 131)  # Crear un array de 0 a 131
w = np.random.normal(0, 1, len(n)) # Generar ruido blanco con media 0 y desviación estándar 1
x = w  # Asignar el ruido blanco a x
aCoef = [1, -2.7377, 3.7476, -2.6293, 0.9224] # Coeficientes del filtro AR (AutoRegresivo)
z = signal.lfilter([1], aCoef, w)   # Aplicar el filtro AR a la señal x


# Inicializar listas para almacenar los valores de AIC, C y FPE
AIC = []
C = []
FPE = []

# Calcular los coeficientes AR y los criterios para cada orden de modelo
for o in range(1, 16):
    a, rho, _ = spectrum.aryule(x, o)  # Estimar los coeficientes AR usando el método de Yule-Walker
    N = len(x)
    AIC.append(spectrum.AIC(N, np.sum(rho), o))  # Calcular el criterio AIC
    C.append(spectrum.MDL(N, np.sum(rho), o))  # Calcular el criterio MDL
    FPE.append(spectrum.FPE(N, np.sum(rho), o))  # Calcular el criterio FPE


# Encontrar las posiciones de los valores mínimos
min_AIC_pos = AIC.index(min(AIC))
min_C_pos = C.index(min(C))
min_FPE_pos = FPE.index(min(FPE))

# Crear subgráficos
fig = make_subplots(rows=1, cols=3, subplot_titles=("AIC", "C", "FPE"))

# Añadir trazas para AIC
fig.add_trace(
    go.Scatter(x=list(range(1, 16)), y=AIC, mode="lines+markers", name="AIC"),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=[min_AIC_pos + 1],
        y=[AIC[min_AIC_pos]],
        mode="markers",
        marker=dict(color="red", size=10),
        name="Min AIC",
    ),
    row=1,
    col=1,
)

# Añadir trazas para C
fig.add_trace(
    go.Scatter(x=list(range(1, 16)), y=C, mode="lines+markers", name="C"), row=1, col=2
)
fig.add_trace(
    go.Scatter(
        x=[min_C_pos + 1],
        y=[C[min_C_pos]],
        mode="markers",
        marker=dict(color="green", size=10),
        name="Min C",
    ),
    row=1,
    col=2,
)

# Añadir trazas para FPE
fig.add_trace(
    go.Scatter(x=list(range(1, 16)), y=FPE, mode="lines+markers", name="FPE"),
    row=1,
    col=3,
)
fig.add_trace(
    go.Scatter(
        x=[min_FPE_pos + 1],
        y=[FPE[min_FPE_pos]],
        mode="markers",
        marker=dict(color="blue", size=10),
        name="Min FPE",
    ),
    row=1,
    col=3,
)

# Actualizar el diseño
fig.update_layout(title="AIC, C y FPE Burg", xaxis_title="Orden", yaxis_title="Valor")

# Mostrar el gráfico
fig.show()

### Modelado de espectro mediante métodos MA

Utilice ahora el método directo y el método de Durbin para obtener los coeficientes del filtro de MA que modelice el siguiente proceso

$$ x[n] = w[n] - 1.5857w[n-1] + 1.9208w[n-2] -1.5229w[n-3]+0.9224w[n-4] $$

$N$ será 128 y el número de estimaciones deberá ser 50. 

- 1º El método directo calcúlelo mediante Blackman-Tuckey con una ventana rectangular de -4 a 4 (usar Funcion Correlogram de spectrum)
- 2º El método de Durbin calcúlelo con $q=4$ y $p=32$ (usar spectrum.ma)

In [12]:
import numpy as np  # Importar la biblioteca numpy para manejo de arrays y funciones matemáticas
import plotly.graph_objects as go  # Importar plotly para crear gráficos interactivos
from scipy import signal  # Importar signal de scipy para procesamiento de señales
from plotly.subplots import make_subplots  # Importar make_subplots para crear subgráficos
import spectrum  # Importar la biblioteca spectrum para análisis espectral
from spectrum.tools import cshift  # Importar cshift de spectrum.tools

# Generar la señal x(n)
np.random.seed(120)  # Fijar la semilla para reproducibilidad
n =  # Crear un array de 0 a 131
w =   # Generar ruido blanco con media 0 y desviación estándar 1 para 50 realizaciones
x =   # Asignar el ruido blanco a x
bCoef =  # Coeficientes del filtro AR (AutoRegresivo)
x =  # Aplicar el filtro AR a la señal x

# Inicializar listas para almacenar las PSD (Densidad Espectral de Potencia)
psdDi = []
psdDu = []

# Calcular las PSD usando los métodos Directo y Durbin para cada realización
for i in range(50):
    ## Método Directo
    psdAux = spectrum.pcorrelogram().psd  # Estimar el PSD usando el método de correlograma
    psd = np.abs(psdAux)  # Tomar el valor absoluto del PSD
    psdDi.append(psd)  # Almacenar el PSD estimado
    ## Método Durbin
    psdDu.append(np.abs(spectrum.pma().psd))  # Estimar el PSD usando el método de Durbin y almacenar el valor absoluto

# Calcular las PSD medias
psdDiMean = # Media de las PSD estimadas por el método Directo y convertir a dB
psdDuMean =  # Media de las PSD estimadas por el método Durbin y convertir a dB

# Frecuencia
freqs = np.linspace(0, 1, 513)  # Crear un array de frecuencias de 0 a 1

# Crear la figura con Plotly
fig = make_subplots(
    rows=1,  # Número de filas de subgráficos
    cols=2,  # Número de columnas de subgráficos
    subplot_titles=("Directo", "Durbin"),  # Títulos de los subgráficos
)

# Añadir trazas para cada método
for i in range(50):
    fig.add_trace(
        go.Scatter(
            x=freqs,  # Asignar el array de frecuencias al eje x
            y=,  # Convertir el PSD a dB y asignarlo al eje y
            mode="lines",  # Modo de trazado en líneas
            name=f"Realización {i+1}",  # Nombre de la serie
            showlegend=False,  # No mostrar la leyenda para estas trazas
            line=dict(color="blue", width=0.1),  # Estilo de línea
        ),
        row=1,  # Fila del subgráfico
        col=1,  # Columna del subgráfico
    )
    fig.add_trace(
        go.Scatter(
            x=freqs,  # Asignar el array de frecuencias al eje x
            y=,  # Convertir el PSD a dB y asignarlo al eje y
            mode="lines",  # Modo de trazado en líneas
            name=f"Realización {i+1}",  # Nombre de la serie
            showlegend=False,  # No mostrar la leyenda para estas trazas
            line=dict(color="red", width=0.1),  # Estilo de línea
        ),
        row=1,  # Fila del subgráfico
        col=2,  # Columna del subgráfico
    )

# Añadir trazas para las PSD medias
fig.add_trace(
    go.Scatter(
        x=freqs,  # Asignar el array de frecuencias al eje x
        y=psdDiMean,  # Asignar la PSD media al eje y
        mode="lines",  # Modo de trazado en líneas
        name="Directo",  # Nombre de la serie
        line=dict(color="blue", width=2),  # Estilo de línea
    ),
    row=1,  # Fila del subgráfico
    col=1,  # Columna del subgráfico
)
fig.add_trace(
    go.Scatter(
        x=freqs,  # Asignar el array de frecuencias al eje x
        y=psdDuMean,  # Asignar la PSD media al eje y
        mode="lines",  # Modo de trazado en líneas
        name="Durbin",  # Nombre de la serie
        line=dict(color="red", width=2),  # Estilo de línea
    ),
    row=1,  # Fila del subgráfico
    col=2,  # Columna del subgráfico
)

# Mostrar la figura
fig.show()

SyntaxError: invalid syntax (2069013952.py, line 10)

### Modelado de Espectros mediante modelos ARMA

En esta última parte guiada de la práctica, es necesario vamos a realizar el mismo procesamiento con la una señal generada mediante 

$$ x[n] = w[n] - 1.5857w[n-1] + 1.9208w[n-2] -1.5229w[n-3]+0.9224w[n-4]$$

El núermo de muestras será 128

In [ ]:
import numpy as np  # Importar la biblioteca numpy para manejo de arrays y funciones matemáticas
import spectrum  # Importar la biblioteca spectrum para análisis espectral
import plotly.graph_objects as go  # Importar plotly para crear gráficos interactivos
import scipy.signal as signal  # Importar signal de scipy para procesamiento de señales

# Generar la señal x(n)
np.random.seed(120)  # Fijar la semilla para reproducibilidad
n = np.arange(0, 128)  # Crear un array de 0 a 128
w = np.random.normal(0, 1, len(n))  # Generar ruido blanco con media 0 y desviación estándar 1
bCoef = [1, -1.5857, 1.9208, -1.5229, 0.9224] # Coeficientes del filtro AR (AutoRegresivo)
aCoef = 1  # Coeficiente del denominador del filtro (en este caso, 1)
x = signal.lfilter(bCoef, [1], w) # Aplicar el filtro AR a la señal x


# Estimar los parámetros del modelo ARMA (orden AR=4, MA=8)
a, b, sigma = spectrum.arma_estimate(x, P=4, Q=8, lag=20)  # Estimar los coeficientes ARMA y sigma
psd = spectrum.arma2psd(A=a, B=b, rho=sigma, NFFT=1024)  # Calcular la PSD a partir de los coeficientes ARMA
psd = psd[:512]  # Tomar los primeros 512 valores de la PSD

# Crear la gráfica de la PSD usando Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=freqs, y=psd, mode="lines", name="Modelo ARMA"))

fig.update_layout(
    title="PSD de la Señal usando el Modelo ARMA",  # Título del gráfico
    xaxis_title="Frecuencia Normalizada",  # Título del eje x
    yaxis_title="Densidad Espectral de Potencia (dB)",  # Título del eje y
)

fig.show()  # Mostrar la figura

### Procesado de señal muscular 

Utilizando las señales que se le proporcionan para esta práctica, compare el funcionamiento del periodograma de welch con cada uno de los modelos presentados durante el desarrollo.

In [ ]:
import numpy as np  # Importar la biblioteca numpy para manejo de arrays y funciones matemáticas
import matplotlib.pyplot as plt  # Importar matplotlib para crear gráficos
from scipy.signal import welch  # Importar welch de scipy para calcular el periodograma de Welch
from spectrum import Periodogram, arma2psd, ma, aryule  # Importar funciones de spectrum para estimación de modelos
import plotly.graph_objects as go  # Importar plotly para crear gráficos interactivos
from plotly.subplots import make_subplots  # Importar make_subplots para crear subgráficos

# Cargar las señales de las actividades desde archivos de texto
Bowing = np.loadtxt("./Actividades/Bowing.txt")
Clapping = np.loadtxt("./Actividades/Clapping.txt")
Handshaking = np.loadtxt("./Actividades/Handshaking.txt")
Hugging = np.loadtxt("./Actividades/Hugging.txt")
Jumping = np.loadtxt("./Actividades/Jumping.txt")
Running = np.loadtxt("./Actividades/Running.txt")
Seating = np.loadtxt("./Actividades/Seating.txt")
Standing = np.loadtxt("./Actividades/Standing.txt")
Walking = np.loadtxt("./Actividades/Walking.txt")
Waving = np.loadtxt("./Actividades/Waving.txt")

# Seleccionar una señal para analizar (por ejemplo, Bowing)
signal = Bowing
if signal.ndim > 1:  # Si tiene más de una dimensión
    signal = signal[:, 0]  # Tomamos solo la primera columna
fs = 1000  # Suponiendo una frecuencia de muestreo de 1000 Hz

# Periodograma de Welch
f_welch, Pxx_welch = welch(signal, fs=fs, nperseg=256) # Calcular el periodograma de Welch

# Modelo AR
ar_order = 4  # Orden del modelo AR
AR, rho, k = aryule(signal, ar_order) # Estimar los parámetros del modelo AR
psd_ar = Periodogram(AR, NFFT=1024, sampling=fs)  # Crear el periodograma del modelo AR
psd_ar()  # Calcular la PSD
f_ar = psd_ar.frequencies()  # Obtener las frecuencias
Pxx_ar = psd_ar.psd  # Obtener la PSD

# Modelo MA
ma_order = 4  # Orden del modelo MA
MA, P = ma(signal, ma_order, M=20) # Estimar los parámetros del modelo MA
psd_ma = Periodogram(MA, NFFT=1024, sampling=fs)  # Crear el periodograma del modelo MA
psd_ma()  # Calcular la PSD
f_ma = psd_ma.frequencies()  # Obtener las frecuencias
Pxx_ma = psd_ma.psd  # Obtener la PSDma_estimate(signal, ma_order)

# Modelo ARMA
arma_order = (2, 2)  # Orden del modelo ARMA (AR=2, MA=2)
A, _, _ = aryule(signal, arma_order[0])  # Coeficientes AR
B, _ = ma(signal, arma_order[1], M=20) # Coeficientes MA          # Estimar los coeficientes ARMA
psd_arma = arma2psd(A=A, B=B, NFFT=1024)  # Crear el periodograma del modelo ARMA
#psd_arma()  # Calcular la PSD
f_arma = np.linspace(0, fs/2, len(psd_arma))  # Obtener las frecuencias normalizadas
Pxx_arma = psd_arma[:len(f_arma)]  # Obtener la PSD

# Crear subplots
fig = make_subplots(
    rows=2,  # Número de filas de subgráficos
    cols=2,  # Número de columnas de subgráficos
    subplot_titles=("Periodograma de Welch", "Modelo AR", "Modelo MA", "Modelo ARMA"),  # Títulos de los subgráficos
)

# Añadir trazas para el periodograma de Welch
fig.add_trace(
    go.Scatter(x=f_welch, y=Pxx_welch, mode="lines", name="Welch"), row=1, col=1
)

# Añadir trazas para el modelo AR
fig.add_trace(go.Scatter(x=f_ar, y=Pxx_ar, mode="lines", name="AR"), row=1, col=2)

# Añadir trazas para el modelo MA
fig.add_trace(go.Scatter(x=f_ma, y=Pxx_ma, mode="lines", name="MA"), row=2, col=1)

# Añadir trazas para el modelo ARMA
fig.add_trace(go.Scatter(x=f_arma, y=Pxx_arma, mode="lines", name="ARMA"), row=2, col=2)

# Actualizar el layout
fig.update_layout(title_text="Estimación de PSD usando diferentes métodos", height=800)

# Mostrar la gráfica
fig.show()